## 1. Extract metadata file

In [ ]:
import requests
import json
from requests.auth import HTTPBasicAuth

In [ ]:
BASE_URL = "http://localhost:8080/api/v1"
USERNAME = "admin"      
PASSWORD = "password"     

resp = requests.get(
    f"{BASE_URL}/Metadata",
    auth=HTTPBasicAuth(USERNAME, PASSWORD),
    headers={"Content-Type": "application/json"}
)

if resp.status_code != 200:
    raise Exception(f"Failed to fetch metadata: {resp.status_code} {resp.text}")

metadata = resp.json()

with open("espocrm_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print("Metadata generated: espocrm_metadata.json")

## 2. Explore the Espo metatadata file

In [ ]:
import json

In [ ]:
with open("D:/skymap/phase2/crm-chatbot/docs/espocrm_metadata.json", "r", encoding="utf-8") as f:
    data = json.load(f)

entity_defs_fields = list(data["entityDefs"].keys())

print(entity_defs_fields)

# 3. Example

## 3.1 Library & Config

In [ ]:
import os
import json
import re
import difflib
from typing import Any, Dict, List, Tuple, Optional, TypedDict

import httpx
from pydantic import BaseModel, Field, create_model

import gradio as gr
from langchain_core.tools import StructuredTool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

In [ ]:
OPENAPI_PATH = os.getenv("OPENAPI_PATH", "docs/espocrm_openapi_spec.json")
TOOLS_PATH = os.getenv("TOOLS_JSON", "docs/tools0.json") 
BASE_URL = os.getenv("CRM_BASE_URL", "http://localhost:8080/api/v1")
AUTH_USER = os.getenv("CRM_USER", "admin")
AUTH_PASS = os.getenv("CRM_PASS", "password")
LLM_MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini")

In [ ]:
client = httpx.Client(
    base_url=BASE_URL,
    auth=(AUTH_USER, AUTH_PASS),
    headers={"Content-Type": "application/json"}
)

## 3.2 OpenAPI parsing to Tools

In [ ]:
def load_openapi(path: str) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def json_type_to_py(t: str):
    return {
        "string": (str, None),
        "integer": (int, None),
        "number": (float, None),
        "boolean": (bool, None),
        "array": (list, None),
        "object": (dict, None),
    }.get(t, (str, None))


def method_to_action(method: str, path: str) -> str:
    m = method.upper()
    if m == "POST":
        return "create"
    if m in ("PUT", "PATCH"):
        return "update"
    if m == "DELETE":
        return "delete"
    if m == "GET":
        return "get" if "{" in path and "}" in path else "list"
    return "chat"


def infer_domain(path: str, operation: Dict[str, Any]) -> str:
    tags = operation.get("tags") or []
    if tags:
        return tags[0].lower()
    seg = path.strip("/").split("/")[0] if "/" in path else path.strip("/")
    return (seg or "general").lower()


def _resolve_ref(root: Dict[str, Any], ref: str) -> Dict[str, Any]:
    """Resolve a JSON Pointer like '#/components/schemas/Foo' from the OpenAPI root."""
    if not ref.startswith("#/"):
        raise KeyError(f"Unsupported $ref: {ref}")
    obj = root
    for part in ref.lstrip("#/").split("/"):
        obj = obj[part]
    return obj

def deref_once(obj: Dict[str, Any], root: Dict[str, Any]) -> Tuple[Dict[str, Any], Optional[str]]:
    """If obj is a $ref, resolve once against the OpenAPI root."""
    if isinstance(obj, dict) and "$ref" in obj:
        ref = obj["$ref"]
        return _resolve_ref(root, ref), ref
    return obj, None


def extract_parameters(operation: Dict[str, Any], openapi_root: Dict[str, Any]) -> Dict[str, Any]:
    properties: Dict[str, Any] = {}
    required: List[str] = []

    # 1) parameters (query/path), include refs
    for p in (operation.get("parameters") or []):
        if "$ref" in p:
            p, _ = deref_once(p, openapi_root)

        pname = p.get("name")
        if not pname:
            continue

        preq = p.get("required", False)

        if "schema" in p:
            pschema = p["schema"]
        elif "content" in p and isinstance(p["content"], dict) and p["content"]:
            _, mt_obj = next(iter(p["content"].items()))
            pschema = mt_obj.get("schema", {"type": "string"})
        else:
            pschema = {"type": "string"}

        pres, pref = deref_once(pschema, openapi_root)
        prop = {"description": p.get("description", ""), **pres}
        if pref:
            prop["x-fromRef"] = pref

        properties[pname] = prop
        if preq:
            required.append(pname)

    # 2) requestBody (application/json)
    rb = operation.get("requestBody")
    if rb:
        rb_req = rb.get("required", False)
        content = rb.get("content", {})
        appjson = content.get("application/json", {})
        if appjson:
            s = appjson.get("schema", {})
            sres, sref = deref_once(s, openapi_root)
            if sres.get("type") == "object":
                if sref:
                    sres["x-originalRef"] = sref
                for k, v in (sres.get("properties") or {}).items():
                    vres, vref = deref_once(v, openapi_root)
                    prop = {**vres}
                    if "description" not in prop:
                        prop["description"] = v.get("description", "")
                    if vref:
                        prop["x-fromRef"] = vref
                    properties[k] = prop
                for r in (sres.get("required") or []):
                    if r not in required:
                        required.append(r)
            else:
                properties["body"] = sres
                if rb_req:
                    required.append("body")

    return {"type": "object", "properties": properties, "required": required}



def build_tools_from_openapi(openapi_dict: Dict[str, Any]) -> List[Dict[str, Any]]:
    paths = openapi_dict.get("paths", {})

    tools: List[Dict[str, Any]] = []
    for path, ops in paths.items():
        path_level_params = ops.get("parameters", [])  # path-level params
        for method, op in ops.items():
            if method.upper() not in ("GET", "POST", "PUT", "PATCH", "DELETE"):
                continue

            # Merge path-level + op-level parameters
            merged_params = (path_level_params or []) + (op.get("parameters") or [])
            op_merged = {**op, "parameters": merged_params}

            action = method_to_action(method, path)
            domain = infer_domain(path, op)
            params = extract_parameters(op_merged, openapi_dict)  # pass ROOT

            name = op.get("operationId") or f"{action}_{domain}".replace(" ", "_").lower()
            tool = {
                "name": name,
                "description": op.get("summary") or op.get("description") or f"{action} {domain}",
                "x-endpoint": {"method": method.upper(), "path": path},
                "x-router": {"action": action, "domain": domain},
                "parameters": params,
            }
            if "$ref" in op:
                tool["x-originalRef"] = op["$ref"]
            tools.append(tool)

    return tools

## 3.3 Build Tools

In [ ]:

def build_args_model_from_parameters(name: str, parameters: Dict[str, Any]) -> type[BaseModel]:
    props = parameters.get("properties") or {}
    req = set(parameters.get("required") or [])
    fields = {}
    for fname, fsch in props.items():
        ftype = fsch.get("type", "string")
        py_type, _ = json_type_to_py(ftype)
        desc = fsch.get("description", "")
        default = ... if fname in req else None
        fields[fname] = (py_type, Field(default=default, description=desc))
    if not fields:
        fields["tool_input"] = (dict, Field(default=...))
    return create_model(f"{name}_ArgsModel", **fields)


def normalize_phone(phone: str, default_cc: str = "VN") -> str:
    digits = re.sub(r"\D", "", phone or "")
    if not digits:
        return phone
    if default_cc.upper() == "VN":
        if digits.startswith("0"):
            digits = "84" + digits[1:]
        elif digits.startswith("84"):
            pass
    if not digits.startswith("+"):
        digits = "+" + digits
    just_digits = re.sub(r"\D", "", digits)
    if not (9 <= len(just_digits) <= 15):
        return phone  
    return digits


def make_tool(tool_def: Dict[str, Any]) -> StructuredTool:
    name = tool_def["name"]
    desc = tool_def.get("description", "")
    endpoint = tool_def["x-endpoint"]
    params = tool_def.get("parameters", {"type": "object", "properties": {}})

    ArgsModel = build_args_model_from_parameters(name, params)

    def _run(**data):
        payload = {k: v for k, v in (data or {}).items() if v not in ("", None)}
        method = endpoint["method"]
        path = endpoint["path"]

        path_params = re.findall(r"{([^}]+)}", path)
        if path_params:
            for p in path_params:
                if p not in payload or payload[p] in ("", None):
                    return {"error": f"Thiếu path param '{p}' cho endpoint {method} {path}"}
            for p in path_params:
                path = path.replace("{"+p+"}", str(payload.pop(p)))

        if method == "POST" and path.lower().startswith("/lead"):
            if "lastName" not in payload:
                raw = (payload.pop("name", "") or "").strip()
                if raw:
                    sal_map = {
                        "mr.": "Mr.", "ms.": "Ms.", "mrs.": "Mrs.", "dr.": "Dr.",
                        "ông": "Mr.", "anh": "Mr.", "bà": "Ms.", "chị": "Ms.",
                    }
                    parts = raw.split()
                    lname = raw
                    if parts and parts[0].lower() in sal_map:
                        payload["salutationName"] = sal_map[parts[0].lower()]
                        lname = " ".join(parts[1:]).strip() or parts[-1]
                    payload["lastName"] = lname

        if "phoneNumber" in payload:
            payload["phoneNumber"] = normalize_phone(payload["phoneNumber"], default_cc="VN")

        print(f"[DEBUG] Tool '{name}' → {method} {path} payload: {payload}")

        if method in ["POST", "PATCH", "PUT"]:
            resp = client.request(method, path, json=payload)
        elif method in ["GET", "DELETE"]:
            resp = client.request(method, path, params=payload)
        else:
            resp = client.request(method, path)
        try:
            return resp.json()
        except Exception:
            return {"status": resp.status_code, "text": resp.text}

    return StructuredTool.from_function(func=_run, name=name, description=desc, args_schema=ArgsModel)

In [ ]:
TOOLS_META: List[Dict[str, Any]] = []
if os.path.exists(TOOLS_PATH):
    with open(TOOLS_PATH, "r", encoding="utf-8") as f:
        TOOLS_META = json.load(f)
else:
    if not os.path.exists(OPENAPI_PATH):
        raise FileNotFoundError(f"Neither {TOOLS_PATH} nor {OPENAPI_PATH} exists")
    openapi = load_openapi(OPENAPI_PATH)
    TOOLS_META = build_tools_from_openapi(openapi)

TOOLS_BY_NAME = {t["name"]: t for t in TOOLS_META}
STRUCTURED_TOOLS = {t["name"]: make_tool(t) for t in TOOLS_META}
ACTION_DOMAIN: Dict[Tuple[str, str], List[StructuredTool]] = {}
for t in TOOLS_META:
    key = (t["x-router"]["action"].lower(), t["x-router"]["domain"].lower())
    ACTION_DOMAIN.setdefault(key, []).append(STRUCTURED_TOOLS[t["name"]])

DOMAINS = sorted(set(t["x-router"]["domain"] for t in TOOLS_META))
ACTIONS = ["list", "get", "create", "update", "delete"]


def get_required_fields(tool_name: str) -> Tuple[List[str], Dict[str, Any]]:
    schema = TOOLS_BY_NAME[tool_name].get("parameters") or {}
    return schema.get("required", []) or [], schema.get("properties", {}) or {}

## 3.4 Graph Nodes

In [ ]:
class State(TypedDict, total=False):
    messages: List[BaseMessage]
    action: str
    domain: str
    pending_tool: Optional[str]
    collected_args: Dict[str, Any]
    missing_fields: List[str]
    awaiting_field: Optional[str]
    options: Optional[List[Any]]
    dep_info: Dict[str, Any]
    router_hint: Optional[str]
    _seen: int  # số message đã render ra UI

router_llm = ChatOpenAI(model=LLM_MODEL)

In [ ]:
def extract_json(text: str):
    m = re.search(r"\{.*\}", text, re.S)
    if m:
        return json.loads(m.group(0))
    raise ValueError("No JSON found")


def router_node(state: State) -> State:
    last = state["messages"][-1].content
    prompt = f"""
Bạn là router phân loại yêu cầu CRM.
Đầu vào: "{last}"
Nếu liên quan CRM → chọn 1 action trong {ACTIONS} và 1 domain trong {DOMAINS}.
Nếu không liên quan → action = "chat", domain = "general".
Chỉ trả JSON thuần, không thêm giải thích.
Ví dụ: {{"action":"create","domain":"account"}}
"""
    resp = router_llm.invoke([HumanMessage(content=prompt)])
    try:
        choice = extract_json(resp.content)
        action = choice["action"].lower()
        domain = choice["domain"].lower()
        if action == "get":
            if any(kw in last.lower() for kw in ["id", "mã", "identifier", "by id"]):
                action = "get"
            else:
                action = "list"
        elif action in ["post"]:
            action = "create"
        elif action in ["put", "patch"]:
            action = "update"
        elif action in ["delete"]:
            action = "delete"
        return {**state, "action": action, "domain": domain}
    except Exception:
        return {**state, "action": "chat", "domain": "general"}


def select_tool_node(state: State) -> State:
    key = (state["action"], state["domain"])
    tools = ACTION_DOMAIN.get(key, [])
    if not tools:
        return {**state, "messages": state["messages"] + [AIMessage(content="Không tìm thấy tool phù hợp cho action/domain.")]} 
    llm = ChatOpenAI(model=LLM_MODEL).bind_tools(tools)
    last_user = state["messages"][-1].content
    system_msg = f"""
Bạn là trợ lý CRM.
- Người dùng muốn {state['action']} {state['domain']}.
- BẮT BUỘC gọi tool phù hợp.
- Mapping phổ biến:
  * "tên là ..." → name (Account) hoặc lastName (Lead)
  * "email ..." | "mail ..." → emailAddress
  * SĐT → phoneNumber
- Tránh gửi khóa rỗng "".
- Với Lead: ưu tiên lastName; suy luận salutationName từ "ông/anh/bà/chị".
"""
    resp = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=last_user)])
    if hasattr(resp, "tool_calls") and resp.tool_calls:
        for call in resp.tool_calls:
            print("[DEBUG] Tool called:", call["name"])
            print("[DEBUG] Args generated:", call["args"])
    return {**state, "messages": state["messages"] + [resp]}


def dependency_checker_node(state: State) -> State:
    last = state["messages"][-1]
    msgs = state["messages"]

    if hasattr(last, "tool_calls") and last.tool_calls:
        call = last.tool_calls[0]
        tool_name, args = call["name"], call["args"]
        req, props = get_required_fields(tool_name)

        # --- NEW: resolve ID by name for endpoints that require {id} ---
        endpoint = TOOLS_BY_NAME[tool_name]["x-endpoint"]
        path_pat = endpoint["path"]

        def is_id_like(x: str) -> bool:
            return bool(re.fullmatch(r"[0-9a-fA-F-]{24,36}", str(x or "")))

        if "{" in path_pat and "}" in path_pat:
            if ("id" not in args) or (not is_id_like(args.get("id"))):
                guess_name = str(args.get("id") or args.get("name") or "").strip()
                dom_from_ep = path_pat.strip("/").split("/")[0]
                if guess_name:
                    try:
                        r = client.get(f"/{dom_from_ep}", params={
                            "where[0][type]": "equals",
                            "where[0][attribute]": "name",
                            "where[0][value]": guess_name,
                            "maxSize": 20,
                        })
                        data = r.json()
                        items = data.get("list", data if isinstance(data, list) else [])
                        options = []
                        for it in items:
                            if isinstance(it, dict) and it.get("id"):
                                label = it.get("name") or it.get("id")
                                options.append(f"{label} | {it['id']}")
                        if len(options) == 1:
                            args["id"] = options[0].split("|", 1)[1].strip()
                        elif len(options) > 1:
                            msgs.append(AIMessage(content=f"Tìm thấy nhiều {dom_from_ep} tên '{guess_name}'. Chọn một:\n" + "\n".join(options)))
                            return {**state, "messages": msgs, "pending_tool": tool_name,
                                    "collected_args": args, "missing_fields": ["id"],
                                    "awaiting_field": "id", "options": options}
                        else:
                            msgs.append(AIMessage(content=f"Không tìm thấy {dom_from_ep} tên '{guess_name}'. Nhập ID chính xác hoặc tên khác."))
                            return {**state, "messages": msgs, "pending_tool": tool_name,
                                    "collected_args": args, "missing_fields": ["id"],
                                    "awaiting_field": "id", "options": None}
                    except Exception as e:
                        msgs.append(AIMessage(content=f"Lỗi tìm ID cho {dom_from_ep}: {e}"))
                        return {**state, "messages": msgs, "pending_tool": tool_name,
                                "collected_args": args, "missing_fields": ["id"],
                                "awaiting_field": "id", "options": None}

        # Required missing?
        missing = [f for f in req if f not in args or args[f] in ("", None)]
        if missing:
            msgs.append(AIMessage(content=f"Cần cung cấp thêm: {', '.join(missing)}"))
            return {**state, "messages": msgs, "pending_tool": tool_name, "collected_args": args, "missing_fields": missing, "awaiting_field": None, "options": None}

        # Enum validate
        invalid_fields = []
        for fname, fsch in props.items():
            if "enum" in fsch and fname in args:
                if args[fname] not in fsch["enum"]:
                    invalid_fields.append((fname, fsch["enum"]))
        if invalid_fields:
            fname, enumv = invalid_fields[0]
            msgs.append(AIMessage(content=f"Giá trị '{fname}' không hợp lệ. Hãy chọn một trong: {', '.join(map(str, enumv))}"))
            return {**state, "messages": msgs, "pending_tool": tool_name, "collected_args": args, "missing_fields": [fname], "awaiting_field": fname, "options": list(enumv)}

        # Cross-API: Country fields → list AddressCountry; don't block on empty
        dep_info = {}
        for fname in list(args.keys()):
            low = fname.lower()
            if low.endswith("country") or low == "addresscountry":
                try:
                    r = client.get("/AddressCountry", params={"maxSize": 1000})
                    data = r.json()
                    if isinstance(data, dict) and "list" in data:
                        options_raw = data["list"]
                    elif isinstance(data, list):
                        options_raw = data
                    else:
                        options_raw = []
                    options = []
                    for item in options_raw:
                        if isinstance(item, dict):
                            options.append(item.get("name") or item.get("id") or json.dumps(item, ensure_ascii=False))
                        else:
                            options.append(str(item))
                except Exception:
                    options = []

                if not options:
                    continue  # accept free-text if server has no list

                if str(args.get(fname, "")).strip() not in options:
                    msgs.append(AIMessage(content=f"Giá trị {fname} chưa khớp. Chọn một trong: {', '.join(options[:20])}"))
                    dep_info[fname] = {"domain": "addresscountry", "options": options}
                    return {**state, "messages": msgs, "pending_tool": tool_name, "collected_args": args,
                            "missing_fields": [fname], "awaiting_field": fname, "options": options, "dep_info": dep_info}

        return {**state, "messages": msgs, "pending_tool": tool_name, "collected_args": args, "missing_fields": []}

    return state


def collector_node(state: State) -> State:
    last_human = None
    for m in reversed(state["messages"]):
        if isinstance(m, HumanMessage):
            last_human = m.content
            break
    if not last_human:
        return state

    tool_name = state.get("pending_tool")
    if not tool_name:
        return state

    req, props = get_required_fields(tool_name)
    args = dict(state.get("collected_args", {}))
    awaiting = state.get("awaiting_field")
    text = last_human.strip()

    # helpers
    def extract_country(text: str, options: Optional[List[str]] = None) -> Optional[str]:
        t = text.lower()
        alias = {
            "việt nam": "Vietnam", "vietnam": "Vietnam",
            "usa": "United States", "united states": "United States",
            "nhật": "Japan", "japan": "Japan",
            "hàn": "Korea", "korea": "Korea",
            "thái": "Thailand", "thailand": "Thailand",
        }
        for k, v in alias.items():
            if k in t:
                return v
        if options:
            cand = difflib.get_close_matches(text, options, n=1, cutoff=0.5)
            if cand:
                return cand[0]
        if any(x in t for x in ["hà nội", "hanoi", "hà đông"]):
            return "Vietnam"
        parts = text.split()
        return parts[-1] if parts else None

    if awaiting:
        if awaiting.lower().startswith("id"):
            # allow either raw id or "Label | id"
            raw = text
            if "|" in raw:
                raw = raw.split("|", 1)[1].strip()
            args[awaiting] = raw
        elif "country" in awaiting.lower():
            val = extract_country(text, state.get("options"))
            if val:
                args[awaiting] = val
        else:
            args[awaiting] = text
    else:
        for f in req:
            if f not in args:
                if f.lower() in ("email", "emailaddress"):
                    m = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", text)
                    if m:
                        args[f] = m.group(0)
                elif f.lower() in ("phone", "phonenumber", "mobile"):
                    m = re.search(r"\+?\d[\d\s\-]{6,}\d", text)
                    if m:
                        args[f] = re.sub(r"[^\d+]", "", m.group(0))
                else:
                    m = re.search(fr"{re.escape(f)}\s*[:=]\s*(.+)$", text, re.I)
                    if m:
                        args[f] = m.group(1).strip()

    missing = [f for f in req if f not in args or args[f] in ("", None)]
    return {**state, "collected_args": args, "missing_fields": missing, "awaiting_field": None, "options": None}


def executor_node(state: State) -> State:
    tool_name = state.get("pending_tool")
    args = state.get("collected_args", {})
    msgs = state["messages"]

    if not tool_name:
        return state

    req, _ = get_required_fields(tool_name)
    missing = [f for f in req if f not in args or args[f] in ("", None)]
    if missing:
        msgs.append(AIMessage(content=f"Bạn vẫn cần cung cấp: {', '.join(missing)}"))
        return {**state, "messages": msgs}

    tool = STRUCTURED_TOOLS.get(tool_name)
    if not tool:
        msgs.append(AIMessage(content=f"Không tìm thấy tool {tool_name}"))
        return {**state, "messages": msgs}

    result = tool.invoke(args)
    msgs.append(ToolMessage(content=json.dumps(result, ensure_ascii=False), tool_call_id=tool_name))

    # Hint khi 404 hoặc error
    if isinstance(result, dict) and (result.get("status") == 404 or result.get("error")):
        msgs.append(AIMessage(content="Không tìm thấy bản ghi hoặc thiếu tham số. Bạn có thể nhập 'id: <id hoặc Tên>' để thử lại."))

    msgs.append(AIMessage(content=f"Đã thực thi {tool_name} với kết quả trên."))
    return {**state, "messages": msgs, "pending_tool": None, "collected_args": {}, "missing_fields": [], "awaiting_field": None, "options": None}


def chat_node(state: State) -> State:
    tool_name = state.get("pending_tool")
    missing = state.get("missing_fields") or []
    options = state.get("options")
    domain = state.get("domain")
    action = state.get("action")

    if not tool_name:
        return state

    if options and len(missing) == 1:
        field = missing[0]
        content = f"Mình đang {action} {domain}. Hãy chọn '{field}' trong: {', '.join(map(str, options[:20]))}"
    elif missing:
        content = f"Để {action} {domain}, cần thêm: {', '.join(missing)}. Bạn nhập theo dạng: {missing[0]}: <giá trị>."
    else:
        content = f"Đang chuẩn bị thực thi {tool_name}. Nếu cần thay đổi thông tin, hãy nhập lại."

    return {**state, "messages": state["messages"] + [AIMessage(content=content)]}

# Build graph
builder = StateGraph(State)

builder.add_node("router", router_node)
builder.add_node("select_tool", select_tool_node)
builder.add_node("dependency_checker", dependency_checker_node)
builder.add_node("collector", collector_node)
builder.add_node("executor", executor_node)
builder.add_node("chat", chat_node)

builder.set_entry_point("router")
builder.add_edge("router", "select_tool")
builder.add_edge("select_tool", "dependency_checker")


def dep_route(s: State):
    if s.get("pending_tool"):
        if s.get("missing_fields") or s.get("awaiting_field") or s.get("options"):
            return "chat"
        return "executor"
    return END

builder.add_conditional_edges("dependency_checker", dep_route, {"chat": "chat", "executor": "executor", END: END})

builder.add_edge("executor", END)
builder.add_edge("chat", END)

graph = builder.compile()


C:\Users\Dell\AppData\Local\Temp\ipykernel_12252\434073824.py:748: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=620, show_label=False)
d:\miniconda3\envs\skymap\Lib\site-packages\gradio\layouts\column.py:59: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[DEBUG] Tool called: list_lead
[DEBUG] Args generated: {}
[DEBUG] Tool 'list_lead' → GET /Lead payload: {}
[DEBUG] Tool called: create_lead
[DEBUG] Args generated: {'lastName': 'HAHAHAHAH', 'emailAddress': 'datmieu2004@gmail.com', 'addressStreet': 'Tổ 5 Nhân Huệ Đồng Mai', 'addressCity': 'Hà Đông', 'addressState': 'Hà Nội'}
[DEBUG] Tool 'create_lead' → POST /Lead payload: {'lastName': 'HAHAHAHAH', 'addressStreet': 'Tổ 5 Nhân Huệ Đồng Mai', 'addressCity': 'Hà Đông', 'addressState': 'Hà Nội', 'emailAddress': 'datmieu2004@gmail.com'}
[DEBUG] Tool called: list_account
[DEBUG] Args generated: {}
[DEBUG] Tool 'list_account' → GET /Account payload: {}


## 3.5 Web UI

In [ ]:
def init_state() -> State:
    return {
        "messages": [],
        "action": "",
        "domain": "",
        "pending_tool": None,
        "collected_args": {},
        "missing_fields": [],
        "awaiting_field": None,
        "options": None,
        "dep_info": {},
        "_seen": 0,
    }


def format_new_ai_messages(state: State, since: int) -> str:
    # Lấy AIMessage mới từ chỉ số 'since'
    new = []
    for m in state["messages"][since:]:
        if isinstance(m, AIMessage):
            new.append(m.content)
        elif isinstance(m, ToolMessage):
            # hiển thị gọn ToolMessage
            try:
                data = json.loads(m.content)
                pretty = json.dumps(data, ensure_ascii=False, indent=2)
            except Exception:
                pretty = m.content
            new.append(f"```json\n{pretty}\n```")
    return "\n\n".join(new).strip() or "(ok)"


def respond(user_msg: str, chat_history: List[Tuple[str, str]], state: State):
    if state is None or not isinstance(state, dict) or "messages" not in state:
        state = init_state()

    # Nếu đang đợi user cung cấp field → collector trước, rồi executor nếu đủ
    if state.get("pending_tool") and (state.get("missing_fields") or state.get("awaiting_field")):
        state["messages"].append(HumanMessage(content=user_msg))
        state = collector_node(state)
        req, _ = get_required_fields(state.get("pending_tool"))
        missing = [f for f in req if f not in state.get("collected_args", {}) or state["collected_args"][f] in ("", None)]
        state["missing_fields"] = missing
        if missing:
            state = chat_node(state)
        else:
            state = executor_node(state)
    else:
        # turn bình thường: router → select_tool → dependency_checker → (chat|executor)
        state["messages"].append(HumanMessage(content=user_msg))
        state = graph.invoke(state)

    # Chuẩn bị trả lời mới
    new_text = format_new_ai_messages(state, state.get("_seen", 0))
    state["_seen"] = len(state["messages"])

    chat_history = chat_history + [(user_msg, new_text)]

    # debug panel info
    debug = {
        "action": state.get("action"),
        "domain": state.get("domain"),
        "pending_tool": state.get("pending_tool"),
        "collected_args": state.get("collected_args"),
        "missing_fields": state.get("missing_fields"),
        "awaiting_field": state.get("awaiting_field"),
    }
    return chat_history, state, json.dumps(debug, ensure_ascii=False, indent=2)


with gr.Blocks(title="CRM Chat — LangGraph") as demo:
    gr.Markdown("""
    # CRM Chat (LangGraph)
    Chat điều khiển luồng CRUD theo OpenAPI → tools. Điền thiếu trường, validate enum, và gọi chéo API (ví dụ AddressCountry).
    """)

    with gr.Row():
        chatbot = gr.Chatbot(height=620, show_label=False)
        with gr.Column(scale=0.5):
            debug_json = gr.Code(label="Debug state", language="json", interactive=False, value="{}")

    msg = gr.Textbox(placeholder="Nhập yêu cầu… ví dụ: Tạo Lead tên bà Thu, email thu@example.com, addressCountry Vietnam")
    state_store = gr.State(init_state())

    def clear_all():
        return [], init_state(), "{}", ""

    submit = gr.Button("Gửi", variant="primary")
    clear_btn = gr.Button("Clear")

    submit.click(respond, inputs=[msg, chatbot, state_store], outputs=[chatbot, state_store, debug_json]).then(lambda: "", None, msg)
    msg.submit(respond, inputs=[msg, chatbot, state_store], outputs=[chatbot, state_store, debug_json]).then(lambda: "", None, msg)
    clear_btn.click(clear_all, inputs=None, outputs=[chatbot, state_store, debug_json, msg])

if __name__ == "__main__":
    demo.launch()